In [1]:
import pandas as pd
import numpy as np
import glob
import gc
import os
import math
import keras
from keras.layers import CuDNNLSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input, GlobalMaxPooling2D
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate, Lambda, Concatenate
from keras.callbacks import LambdaCallback
from keras.models import Model
from keras import regularizers
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2

gc.enable()

Using TensorFlow backend.


In [2]:
from modules import clr, LRFinder
from modules.utils import DataGenerator

In [3]:
import tensorflow as tf
import random

def seed_everything(seed=2019):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                                  inter_op_parallelism_threads=1)
    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    
seed_everything()


In [4]:
train = pd.read_csv(r'C:\Users\zheng\Desktop\ongoing_DL\shopee\data\train.csv')

# load data

In [5]:
print(train.head())

    itemid                                              title  Category  \
0   307504               nyx sex bomb pallete natural palette         0   
1   461203  etude house precious mineral any cushion pearl...         1   
2  3592295                           milani rose powder blush         2   
3  4460167                etude house baby sweet sugar powder         3   
4  5853995       bedak revlon color stay aqua mineral make up         3   

                                          image_path  
0  beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg  
1  beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg  
2  beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg  
3  beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg  
4  beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg  


In [6]:
mobile_class = np.arange(31, 58)
id_mobile_class = np.append(np.arange(31, 35), np.arange(36, 58))

In [7]:
batchsize = 128
height = 299
width = 299
img_shape = (height, width, 3)
seed = 2019
path = "D:/shopee_resized_data/"

In [8]:
trn_index_path = r"C:\Users\zheng\Desktop\ongoing_DL\shopee\trn_index\mobile"
val_index_path = r"C:\Users\zheng\Desktop\ongoing_DL\shopee\val_index\mobile"

trn_index_np = glob.glob(trn_index_path + "/*")
val_index_np = glob.glob(val_index_path + "/*")

In [9]:
trn_index = {}
val_index = {}

for i in trn_index_np:
    np_array = np.load(i)
    class_ = int(i.split('\\')[-1].split('.')[0])
    trn_index[class_] = np_array

for i in val_index_np:
    np_array = np.load(i)
    class_ = int(i.split('\\')[-1].split('.')[0])
    val_index[class_] = np_array

In [10]:
img_paths_trn = list(train.iloc[:, 3])

In [21]:
def read_img(path):
    image = cv2.imread(path)
    # add noise
    
    
    if image.size != (299, 299, 3):
        image = cv2.resize(image, (299, 299))
    return image

In [12]:
def generate_pairs(id_classes, all_classes, idx_dict):
    pairs = []
    labels = []
    for i in tqdm(id_classes):
        idx_array_1 = idx_dict[i]
        length = idx_array_1.shape[0]
        for j in all_classes:
            idx_array_2 = idx_dict[j]
            length_2 = idx_array_2.shape[0]
            if j == i:
                for count in range(2000):
                    pair = []
                    pair.append(img_paths_trn[idx_array_1[np.random.randint(length)]])
                    pair.append(img_paths_trn[idx_array_1[np.random.randint(length)]])
                    labels.append(1)
                    pairs.append(pair)
            else:
                for count in range(100):
                    pair = []
                    pair.append(img_paths_trn[idx_array_1[np.random.randint(length)]])
                    pair.append(img_paths_trn[idx_array_2[np.random.randint(length_2)]])
                    labels.append(0)
                    pairs.append(pair)
    return np.array(pairs), np.array(labels)

In [53]:
class Siamese_DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, img_pairs, labels, batch_size=128, dim=(299, 299), n_channels=3,
                 shuffle=True, path=''):
        'Initialization'
        self.img_pairs = img_pairs
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.path = path
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.img_pairs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.img_pairs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        # X = np.empty((2, self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, 1), dtype=int)

        # Generate data
        batch_1 = []
        batch_2 = []
        X = [batch_1, batch_2]
        for count, idx in enumerate(indexes):
            pair_path = self.img_pairs[idx]
            for num, path_ in enumerate(pair_path):
                image = read_img(self.path+path_) #augmentation and resize
                image = image/255
                X[num].append(image)

            y[count] = self.labels[idx]
        
        return [np.array(X[0]), np.array(X[1])], np.array(y)

In [51]:
pairs, labels = generate_pairs(id_mobile_class, mobile_class, trn_index)

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 43.74it/s]


In [15]:
labels.shape

(119600,)

# build model

In [16]:
sgd = keras.optimizers.SGD(lr=1e-2, momentum=0.9)
input_tensor=Input(shape=(height, width, 3))

In [17]:
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
# model = FC_layer(backbone_model, mobile_class)

In [18]:
def build_model(base_model, optim, activation='sigmoid', input_tensor=None):
    ##############
    # BRANCH MODEL
    ##############
    x = GlobalMaxPooling2D()(base_model.output)
    branch_model = Model(inputs=base_model.input, outputs=x)

    ############
    # HEAD MODEL
    ############
    mid = 32
    xa_inp = Input(shape=branch_model.output_shape[1:])
    xb_inp = Input(shape=branch_model.output_shape[1:])
    x1 = Lambda(lambda x: x[0] * x[1])([xa_inp, xb_inp])
    x2 = Lambda(lambda x: x[0] + x[1])([xa_inp, xb_inp])
    x3 = Lambda(lambda x: K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x4 = Lambda(lambda x: K.square(x))(x3)
    x = Concatenate()([x1, x2, x3, x4])
    x = Reshape((4, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x = Conv2D(mid, (4, 1), activation='relu', padding='valid')(x)
    x = Reshape((branch_model.output_shape[1], mid, 1))(x)
    x = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x = Flatten(name='flatten')(x)

    # Weighted sum implemented as a Dense layer.
    x = Dense(1, use_bias=True, activation=activation, name='weighted-average')(x)
    head_model = Model([xa_inp, xb_inp], x, name='head')

    ########################
    # SIAMESE NEURAL NETWORK
    ########################
    # Complete model is constructed by calling the branch model on each input image,
    # and then the head model on the resulting 512-vectors.
    img_a = Input(shape=img_shape)
    img_b = Input(shape=img_shape)
    xa = branch_model(img_a)
    xb = branch_model(img_b)
    x = head_model([xa, xb])
    model = Model([img_a, img_b], x)
    model.compile(optim, loss='binary_crossentropy', metrics=['binary_crossentropy', 'acc'])
    return model, branch_model, head_model

In [39]:
model, branch_model, head_model = build_model(base_model, sgd)

In [13]:
# freeze the layers first
for layer in backbone_model.layers:
    layer.trainable = False

In [14]:
#kernel_regularizer: instance of keras.regularizers.Regularizer
#bias_regularizer: instance of keras.regularizers.Regularizer
#activity_regularizer: instance of keras.regularizers.Regularizer

def FC_layer(cv_input, classes):
    cv_input_dense = Flatten()(cv_input.output)
    cv_input_dense = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(cv_input_dense)
    main_ = BatchNormalization()(cv_input_dense)
    main_ = Dropout(0.5)(main_)
    main_ = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(main_)
    main_ = BatchNormalization()(main_)
    main_ = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01))(main_)
    main_ = BatchNormalization()(main_)
    output = Dense(len(classes), activation='softmax')(main_)
    
    model = Model(inputs=cv_input.input, outputs=output)
    return model

In [15]:
trn_index_ = trn_index.copy()
np.random.shuffle(trn_index_)
val_index_ = val_index.copy()
np.random.shuffle(val_index_)

In [37]:
model = FC_layer(backbone_model, mobile_class)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lr_finder = LRFinder.LRFinder(model)
lr_finder.find(training_generator, img_paths, start_lr=0.0000001, end_lr=100, epochs=5, batch_size = batchsize)
lr_finder.plot_loss(n_skip_beginning=20, n_skip_end=5)
plot.show()

Epoch 1/5
  13/1127 [..............................] - ETA: 1:08:00 - loss: 4.0681 - acc: 0.00 - ETA: 39:36 - loss: 4.0738 - acc: 0.0195 - ETA: 30:06 - loss: 4.0601 - acc: 0.02 - ETA: 36:07 - loss: 4.0705 - acc: 0.02 - ETA: 41:15 - loss: 4.0757 - acc: 0.02 - ETA: 45:22 - loss: 4.0800 - acc: 0.02 - ETA: 48:13 - loss: 4.0864 - acc: 0.02 - ETA: 50:10 - loss: 4.0937 - acc: 0.02 - ETA: 51:52 - loss: 4.1001 - acc: 0.02 - ETA: 52:51 - loss: 4.0937 - acc: 0.02 - ETA: 53:44 - loss: 4.0941 - acc: 0.02 - ETA: 54:35 - loss: 4.0907 - acc: 0.02 - ETA: 55:12 - loss: 4.0937 - acc: 0.0258

KeyboardInterrupt: 

# acutal training

In [17]:
#cyclic_lr = clr.CyclicLR(base_lr=0.001, max_lr=0.006)

In [55]:
params = {'dim': (299, 299),
          'batch_size': 8,
          'n_channels': 3,
          'shuffle': True,
          "path": path
         }

training_generator = Siamese_DataGenerator(pairs, labels, **params)

model.fit_generator(generator = training_generator, 
                    #validation_data = testing_generator,
                    epochs = 1,
                    verbose = 1
                   )

Epoch 1/1
   26/14950 [..............................] - ETA: 6:16:27 - loss: 0.5117 - binary_crossentropy: 0.5117 - acc: 0.75 - ETA: 3:42:07 - loss: 0.6799 - binary_crossentropy: 0.6799 - acc: 0.62 - ETA: 2:50:36 - loss: 0.8804 - binary_crossentropy: 0.8804 - acc: 0.54 - ETA: 2:24:46 - loss: 0.8261 - binary_crossentropy: 0.8261 - acc: 0.56 - ETA: 2:15:37 - loss: 0.7947 - binary_crossentropy: 0.7947 - acc: 0.57 - ETA: 2:12:55 - loss: 0.7975 - binary_crossentropy: 0.7975 - acc: 0.54 - ETA: 2:11:39 - loss: 0.7913 - binary_crossentropy: 0.7913 - acc: 0.50 - ETA: 2:06:16 - loss: 0.7762 - binary_crossentropy: 0.7762 - acc: 0.51 - ETA: 2:05:24 - loss: 0.7724 - binary_crossentropy: 0.7724 - acc: 0.51 - ETA: 2:09:18 - loss: 0.7723 - binary_crossentropy: 0.7723 - acc: 0.51 - ETA: 2:06:47 - loss: 0.7810 - binary_crossentropy: 0.7810 - acc: 0.48 - ETA: 2:08:54 - loss: 0.7849 - binary_crossentropy: 0.7849 - acc: 0.46 - ETA: 2:08:48 - loss: 0.7893 - binary_crossentropy: 0.7893 - acc: 0.46 - ETA: 2:

KeyboardInterrupt: 